In [13]:
# Importing required libraries

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import API
from tweepy import Cursor

import socket
import re
import sys

import pyspark
import json

import twitter_creds

In [14]:
# Creating authenticated connection with twitter

auth = OAuthHandler(twitter_creds.API_TOKEN, twitter_creds.API_TOKEN_SECRET)
auth.set_access_token(twitter_creds.ACCESS_TOKEN, twitter_creds.ACCESS_TOKEN_SECRET)

api = API(auth)

In [6]:
client = socket.socket()						# Create a client socket
host = '127.0.0.1'									# Define host
port = 10000										# Define port

print('Waiting for connection')
try:
    client.connect((host, port))				# Connect to host and port
except socket.error as e:
    print(str(e))


Waiting for connection
[WinError 10061] No connection could be made because the target machine actively refused it


In [24]:
# Creating a stream listener to listen for incoming tweets

class StdOutListener(StreamListener):
    def __init__(self):
        super(StdOutListener, self).__init__()
        self.rec_tweets = 0
    
    def on_data(self, data):
        all_data = json.loads(data) # Create a dictionary of incoming streams

        username, tweet = extractTweets(all_data)

        if all_data["lang"] == 'en' and "retweeted_status" not in all_data: # filter english and original tweets (tweets that are not re-tweeted)
            self.rec_tweets += 1
            if self.rec_tweets <= 500:
                print("Tweet #{}: \n".format(self.rec_tweets))
                tweet = cleanTweets(tweet)
#                 send_data_to_port(tweet)
                print(tweet)
                return True
            else:
                print("{} Tweets Streamed".format(self.rec_tweets-1))
                send_data_to_port("ok_com")
                return False
        
    
    def on_error(self, status): # this function prints any error encountered while receiveing tweets
        if status == 420:
            #returning False in on_error disconnects the stream
            return False

In [34]:
def cleanTweets(tweet):
    tweet = re.sub("@[\w]*", "", tweet)
    tweet = re.sub("https?://[A-Za-z0-9./]*", "", tweet)
    tweet = re.sub("\n", "", tweet)
    tweet = re.sub("(["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "])", "", tweet)
    return tweet

In [31]:
def extractTweets(all_data):
    
    username = all_data["user"]["screen_name"] # get screen_name of the user
    
    if "extended_tweet" in all_data:
        tweet = all_data["extended_tweet"]["full_text"] # Extract full tweets of user if tweets are truncated
    else:
        tweet = all_data["text"] # Extract tweet if it's not truncated

    return username, tweet

In [9]:
'''def filterTweets(all_data, rec_tweets):
    
    val_dict = {}
    
    if all_data["lang"] == 'en' and "retweeted_status" not in all_data: # filter english and original tweets (tweets that are not re-tweeted)
        rec_tweets += 1
        if rec_tweets <= 10:
            print("Tweet #{}: \n".format(rec_tweets))
            send_data_to_port(tweet)
            val_dict["ret_val"  : True, "rec_tweets" : rec_tweets]
            return val_dict
        else:
            print("{} Tweets Streamed".format(rec_tweets-1))
            send_data_to_port("ok_com")
            val_dict["ret_val"  : False, "rec_tweets" : rec_tweets]
            return val_dict'''

'def filterTweets(all_data, rec_tweets):\n    \n    val_dict = {}\n    \n    if all_data["lang"] == \'en\' and "retweeted_status" not in all_data: # filter english and original tweets (tweets that are not re-tweeted)\n        rec_tweets += 1\n        if rec_tweets <= 10:\n            print("Tweet #{}: \n".format(rec_tweets))\n            send_data_to_port(tweet)\n            val_dict["ret_val"  : True, "rec_tweets" : rec_tweets]\n            return val_dict\n        else:\n            print("{} Tweets Streamed".format(rec_tweets-1))\n            send_data_to_port("ok_com")\n            val_dict["ret_val"  : False, "rec_tweets" : rec_tweets]\n            return val_dict'

In [10]:
def send_data_to_port(data):
    client_input = data
    msg = None
    while msg != "ok_com":
        client.send(client_input.encode('utf-8'))
        print("Data sent : " + data)
        break

#     ClientSocket.close()
    
    
    

In [32]:
# Creating a stream object which is used to fetch the incoming tweets

class TwitterStreamer():
    def stream_tweets(self, hash_tag_list):
        listener = StdOutListener() # Creating a listener object
        stream = Stream(auth, listener) # creating a stream object
        stream.filter(track=hash_tag_list) # filtering incoming tweets on the bases of the search string
        

In [35]:
hash_tag_list = ["fall guys", "PS4", "PC"] # the list on the bases of which the streaming tweets will be filtered
fetched_tweets_filename = "tweets.json" # file name where tweets will be stored

ts = TwitterStreamer() # creating a streamer object
ts.stream_tweets(hash_tag_list) # Start streaming the tweets

Tweet #1: 

  hi can I please get fall guys for free for pc. I have been wanting this game since it was released and I loved your previous game BROFORCE also but due to the pandemic, I don't have the budget. it would mean a lot to me 
Tweet #2: 

 Hy, this my last project FanArt fall guys 3D by me ♥️ 
Tweet #3: 

     I had a 360 during the month-early DLC days and it felt great but now am on PS4 and a COD exclusivity deal just seems a bit pointless, all we really got for MW was survival and no one really cares about that
Tweet #4: 

 PC Part Picker knows!
Tweet #5: 

   is not involved with XRP Toolkit, it's developed by  and . You need to update to the latest firmware (1.6.1) and XRP app (2.0.1) using Ledger Live on a PC.
Tweet #6: 

i want mingyu pc mom 
Tweet #7: 

[SG GO] ONEUS “LIVED” (MAKESTAR)Price: $14 per album• NO 2ND PAYMENT REQUIRED• Comes with makestar’s ver b pc• Can choose between sealed/internal tradingDeadline for payment: 2nd Sept 6PM SGTETA: OctoberIf interested, pl

KeyboardInterrupt: 